In [1]:
import numpy as np
from NeuroConn.preprocessing.preprocessing import FmriPreppedDataSet
from nilearn import plotting

from sklearn import svm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nilearn
from nilearn.connectome import sym_matrix_to_vec
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score

import plotly.express as px
import plotly.graph_objects as go

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectPercentile
import scipy.io

import csv

In [2]:
feature_directory = '/Users/emafikre/Desktop/vincent/Features_1000'
file_list = os.listdir(feature_directory)

numpy_files = [f for f in file_list]
numpy_files.sort()

for i, numpy_file in enumerate(numpy_files):
    data = np.load(os.path.join(feature_directory, numpy_file))
    if len(data.shape) == 3:
        data = data[0, :, :]
    
    if np.isnan(data).any():
        print('Missing value(s) found in file:', numpy_file)

    number = str(i+1).zfill(3)

    mat_file = 'subject_{}.mat'.format(number)
    
    scipy.io.savemat(mat_file, {'data': data}, do_compression=True)
    os.rename(mat_file, os.path.join('/Users/emafikre/Desktop/MatlabProject/Schizophrenic/matrices', mat_file))

In [3]:
image = nilearn.datasets.fetch_atlas_schaefer_2018(n_rois=1000)
coordinates, label_names= plotting.find_parcellation_cut_coords(labels_img=image.maps, return_label_names=True)

In [5]:
image.maps

'/Users/emafikre/nilearn_data/schaefer_2018/Schaefer2018_1000Parcels_7Networks_order_FSLMNI152_1mm.nii.gz'

In [10]:
output_file = 'BrainRegions.csv'

# Write data to the CSV file
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['x', 'y', 'z', 'label'])  # Write header row
    for i in range(len(coordinates)):
        x, y, z = coordinates[i]
        label = label_names[i]
        writer.writerow([x, y, z, label])

print(f"Data has been saved to {output_file}.")

Data has been saved to BrainRegions.csv.


In [25]:
dataset_paths = [
    '/Users/emafikre/Desktop/vincent/KTT_participants.tsv',
    '/Users/emafikre/Desktop/vincent/participants_LA5c.tsv',
    '/Users/emafikre/Desktop/vincent/COBRE_control_participants.tsv',
    '/Users/emafikre/Desktop/vincent/COBRE_scz_participants.tsv'
]

datasets = []
for path in dataset_paths:
    dataset = pd.read_csv(path, sep='\t')
    datasets.append(dataset)

for i in range(len(datasets)):
    dataset = datasets[i]

    if 'diag' in dataset.columns:
        dataset.rename(columns={'diag': 'diagnosis'}, inplace=True)
    elif 'dx' in dataset.columns:
        dataset.rename(columns={'dx': 'diagnosis'}, inplace=True)
    elif 'gender' in dataset.columns:
        dataset.rename(columns={'gender': 'sex'}, inplace=True)

    dataset = dataset[['participant_id', 'diagnosis', 'age', 'sex']]

    dataset = dataset.replace(to_replace=4, value="1")
    dataset = dataset.replace(to_replace=0, value="0")
    dataset = dataset.replace(to_replace=["CONTROL"], value="0")
    dataset = dataset.replace(to_replace=["SCHZ"], value="1")
    dataset = dataset.replace(to_replace=["No_Known_Disorder"], value="0")
    dataset = dataset.replace(to_replace=["Schizophrenia_Strict"], value="1")
    dataset = dataset[(dataset['diagnosis'] == '0') | (dataset['diagnosis'] == '1')]
    dataset = dataset.replace(to_replace=["M"], value="1")
    dataset = dataset.replace(to_replace=["F"], value="2")
    dataset = dataset.replace(to_replace=["male"], value="1")
    dataset = dataset.replace(to_replace=["female"], value="2")

    dataset['participant_id'] = dataset['participant_id'].apply(lambda x: 'sub-' + x if not x.startswith('sub-') else x)

    datasets[i] = dataset

y = pd.concat(datasets, ignore_index=True)

filenames = os.listdir('/Users/emafikre/Desktop/vincent/Features')

for participant in y['participant_id']:
    if participant not in filenames:
        y.drop(y[y['participant_id'] == participant].index, inplace = True)

y = y[['diagnosis','diagnosis', 'age', 'sex']]
y.reset_index(drop=True, inplace=True)
y.iloc[:, 1] = y.iloc[:,1].replace({'0': '1', '1': '0'})
y

,diagnosis,diagnosis,age,sex
0,1,0,37,1
1,1,0,29,1
2,1,0,27,2
3,1,0,35,1
4,1,0,44,2
...,...,...,...,...
437,1,0,55,1
438,1,0,40,2
439,1,0,21,1
440,1,0,52,1


In [26]:
y.to_csv('/Users/emafikre/Desktop/MatlabProject/predesign.csv')

In [27]:
csv = pd.read_csv('/Users/emafikre/Desktop/MatlabProject/predesign.csv', header= None)
y_fixed = csv.drop(index=0, columns=0)

In [28]:
mat = y_fixed.values

In [30]:
scipy.io.savemat('design.mat', {'design': mat})